<a href="https://colab.research.google.com/github/Nagendra061192/Llama-2-on-CPU-Machine/blob/main/demo_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install langchain_google_genai

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["GOOGLE_API_KEY"] = getpass.getpass()
os.environ["TAVILY_API_KEY"] = getpass.getpass()

from langchain_google_genai import ChatGoogleGenerativeAI



··········
··········
··········


In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
message=[SystemMessage(content="you are a nice bot, answer everything in brief"),
        HumanMessage(content="hi,how are you buddy"),
    ]




In [ ]:
model.invoke(message)

AIMessage(content="I'm doing well, thank you.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-792c1b64-6633-4aa9-b471-18d619d24b09-0', usage_metadata={'input_tokens': 17, 'output_tokens': 9, 'total_tokens': 26})

In [ ]:
#use output parser

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parser=StrOutputParser()

In [ ]:
response=model.invoke(message)

In [ ]:
parser.invoke(response)

"Hi there! I'm doing well, thanks for asking. I'm here to help answer any questions you may have. Please let me know if there's anything I can assist you with."

In [ ]:
#Chaining with LCEL

In [ ]:
#added two components
chain=model | parser

In [ ]:
chain.invoke(message)

"Thanks for the compliment! I'm doing well, just assisting users like you with their questions. How can I help you today?"

In [ ]:
### Now Lets Start with the Prompting

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
Prompt_Template=ChatPromptTemplate.from_messages(
    [("system","Translate the following into {language}"),
     ("user","{text}")]
)

In [ ]:
prompt= Prompt_Template.invoke({"language":"Hindi","text":"i am learning data science"})

In [ ]:
model.invoke(prompt)

AIMessage(content='मैं डेटा साइंस सीख रहा हूँ', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-b0c9459e-e673-4a82-b64e-dfaa945fcb02-0', usage_metadata={'input_tokens': 11, 'output_tokens': 12, 'total_tokens': 23})

In [ ]:
#Chaining

In [ ]:
Chain= Prompt_Template | model  | parser

In [ ]:
Chain.invoke({"language" :"hindi" , "text" : "i am thirsty" })

'मुझे प्यास लगी है'

In [ ]:
#Agents

In [ ]:
!pip install langchain-community
!pip install langgraph
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain.agents import AgentExecutor

In [ ]:
memory=MemorySaver()

In [ ]:
Search = TavilySearchResults(max_results=5,search_depth="advanced",include_answer=True,include_raw_content=True,include_images=True,)

In [ ]:
tools = [search]

In [ ]:
agent_executor=create_react_agent(model=model,tools=tools,checkpointer=memory)

In [ ]:
config = {"configurable" : {"thread_id": "abc123"}}

In [ ]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content= "Hi i am nag and i live in noida")]},config=config):
  print(chunk)
  print("---")

In [ ]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content= "What is the weather where i live?")]},config):
  print(chunk)
  print("---")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is the weather in noida"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-226c0582-0755-4db1-803e-f9455d498f6f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is the weather in noida'}, 'id': 'd1b1e06e-c5a2-4237-8211-2768bef9b210', 'type': 'tool_call'}], usage_metadata={'input_tokens': 467, 'output_tokens': 26, 'total_tokens': 493})]}}
---
{'tools': {'me